In [4]:
import pickle
import itertools
import open3d as o3d
import numpy as np
from scipy import interpolate
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D
from scipy import interpolate
from scipy import ndimage
import sklearn
from sklearn.decomposition import PCA
import numpy as np
from scipy.optimize import linear_sum_assignment
from scipy.spatial.distance import cdist


In [5]:
def edges_to_lineset(mesh, edges, color):
    ls = o3d.geometry.LineSet()
    ls.points = mesh.vertices
    ls.lines = edges
    colors = np.empty((np.asarray(edges).shape[0], 3))
    colors[:] = color
    ls.colors = o3d.utility.Vector3dVector(colors)
    return ls

def rotation_mat(angle):
    Rx = np.array([[1,0,0],
                 [0, np.cos(angle[0]), -np.sin(angle[0])],
                 [0, np.sin(angle[0]), np.cos(angle[0])]])

    Ry = np.array([[np.cos(angle[1]), 0, np.sin(angle[1])],
                 [0,1,0],
                 [-np.sin(angle[1]), 0, np.cos(angle[1])]])

    Rz = np.array([[np.cos(angle[2]), -np.sin(angle[2]), 0],
                 [np.sin(angle[2]), np.cos(angle[2]), 0],
                 [0,0,1]])
    return Rz@Rx@Ry
angle = [np.pi, np.pi, 0] # pcd.rotate()
#angle = [-np.pi/2, np.pi/2, np.pi/2]
R = rotation_mat(angle)
R_mirror = np.array([[-1,0,0],
                    [0,1,0],
                    [0,0,1]])

def get_com(mesh, edges):
    length = 500
    vertices = np.asarray(mesh.vertices)
    edges = np.asarray(edges)

    for i in range(len(vertices[edges])):
        start = vertices[edges][i][0]/100
        end = vertices[edges][i][1]/100

        _com = end + np.repeat(np.arange(length+1),3).reshape((length+1,3)) * (start - end)/length

        if i == 0:
            com = np.copy(_com)
            continue
        com = np.concatenate([com, _com])
    return com

In [98]:
def get_pts(mesh, edges, max_point_num=50000):
    vertices = np.asarray(mesh.vertices)
    edges = np.asarray(edges)
    dist = 0
    for i in range(len(vertices[edges])):
        start = vertices[edges][i][0]
        end = vertices[edges][i][1]
        dist_part = np.sqrt(np.sum(np.square(start - end)))
        dist += dist_part
    unit_dist = dist/max_point_num
    next_start_length = 0
    next_start_partation = 0
    pts_list = []
    for i in range(len(vertices[edges])):
        start = vertices[edges][i][0]
        end = vertices[edges][i][1]
        next_st_pt = start + (end - start)*next_start_partation
        dist_part = np.sqrt(np.sum(np.square(start - end)))
        dist_nx = np.sqrt(np.sum(np.square(next_st_pt - end)))
        pts_num = int(dist_nx/unit_dist)+1
        pts = next_st_pt + np.repeat(np.arange(pts_num),3).reshape((pts_num,3)) * (end - next_st_pt)/(pts_num-1)
        end_pts = pts[-1]
        next_start_length = pts_num*unit_dist - dist_part + next_start_length
        next_start_partation = next_start_length/dist_part
        pts_list.append(pts)

    pts_array = np.concatenate(pts_list)

    return pts_array

def cal_emd(ind, s_curve_list):
    rng = np.random.default_rng(123)
    mesh = o3d.io.read_triangle_mesh("polygon/quan_e_a/{}.ply".format(ind))
    y_emd = []
    x_area = []
    for i in range(np.max(np.asarray(mesh.cluster_connected_triangles()[0]))+1):
        mesh = o3d.io.read_triangle_mesh("polygon/quan_e_a/{}.ply".format(ind))
        mesh.triangles = o3d.utility.Vector3iVector(np.asarray(mesh.triangles)[np.asarray(mesh.cluster_connected_triangles()[0])==i])

        _, _, area = mesh.cluster_connected_triangles()
        edges = mesh.get_non_manifold_edges(allow_boundary_edges=False)
        com = get_pts(mesh, edges)
        emd_list = []
        for j in range(8):
            x1 = s_curve_list[j]
            #x1 = x1[np.round(np.arange(500) * len(x1)/500).astype("int64")]

            pcd = o3d.geometry.PointCloud()
            pcd.points = o3d.utility.Vector3dVector(x1)
            pcd.rotate(np.linalg.inv(R),(0,0,0))
            pcd.scale(10,(0,0,0))
            x1 = np.asarray(pcd.points)
            x2 = com
            d = cdist(x1, x2)
            assignment = linear_sum_assignment(d)
            emd = d[assignment].sum() / x1.shape[0]
            emd_list.append(emd)
        emd = np.min(emd_list)
        y_emd.append(emd)
        x_area.append(area[0])
    return y_emd, x_area

In [115]:
curve_list=[
    "curves_by/legacy/s_no_hole_leaf.curves"
    "curves_by/fixed/smoothed/s_112_b20_s105_lmin40_lmax_100_o15_one_hole_leaf.curves",
    "curves_by/fixed/smoothed/s_112_b20_s105_lmin40_lmax_100_o15_two_hole_leaf.curves",
    "curves_by/fixed/smoothed/s_112_b20_s105_lmin40_lmax_100_o15_three_hole_leaf.curves",
    "curves_by/fixed/smoothed/s_112_b20_s105_lmin40_lmax_100_o15_four_hole_leaf.curves",
    "curves_by/fixed/smoothed/s_112_b20_s105_lmin40_lmax_100_o15_five_hole_leaf.curves",
    "curves_by/fixed/smoothed/s_112_b20_s100_lmin40_lmax_100_o15_six_hole_leaf.curves"
]

ply_list = [
    "",
    "polygon/fix/hole1/one_hole_leaf.ply",
    "polygon/fix/hole1/two_hole_leaf.ply",
    "polygon/fix/hole1/three_hole_leaf.ply",
    "polygon/fix/hole1/four_hole_leaf.ply",
    "polygon/fix/hole1/five_hole_leaf.ply",
    "polygon/fix/hole1/six_hole_leaf.ply",
]

In [116]:
def cal_emd(hole_num):
    with open(curve_list[hole_num], "rb") as pickle_file:
        s_curve_list = pickle.load(pickle_file)

    s_curve_list = s_curve_list[1:]

    mesh = o3d.io.read_triangle_mesh(ply_list[hole_num])
    mesh.scale(1/100,(0,0,0))
    mesh.triangles = o3d.utility.Vector3iVector(np.asarray(mesh.triangles))
    edges = mesh.get_non_manifold_edges(allow_boundary_edges=False)
    com = get_pts(mesh, edges)

    x1 = np.concatenate([i.T for i in s_curve_list])

    rng = np.random.default_rng()
    rng.shuffle(x1, axis=0)
    x1 = x1[:1500*hole_num]
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(x1)
    pcd.rotate(np.linalg.inv(R),(0,0,0))
    pcd.scale(10,(0,0,0))
    x1 = np.asarray(pcd.points)

    pcd_ = o3d.geometry.PointCloud()
    pcd_.points = o3d.utility.Vector3dVector(com)
    o3d.visualization.draw_geometries([pcd,pcd_])

    x2 = com
    d = cdist(x1, x2)
    assignment = linear_sum_assignment(d)
    emd = d[assignment].sum() / x1.shape[0]
    return emd

In [119]:
cal_emd(2)

0.13777602544748443

In [109]:
emd

0.012479665171195855